# 1. 自动识别

In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if 'MRP' in file:
        mrp = './' + file
    elif '整理好的替代料总表' in file:
        alternate = './' + file
    elif '改价表' in file:
        price = './' + file
    elif '在途管控表' in file:
        shipping = './' + file
    

print('mrp:            ' + mrp)
print('alternate:      ' + alternate)
print('price:          ' + price)
print('shipping:       ' + shipping)

mrp:            ./US MRP20240318-运营.xlsx
alternate:      ./整理好的替代料总表_20240325_162417.xlsx
price:          ./改价表 3-19.xlsx
shipping:       ./【新老品在途管控表】US-ebay 20240319.xlsx


# 2. 一切以mrp为主

In [3]:
import pandas as pd

In [4]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       dtype=str).fillna('')[['MRP SKU',
                                              '自动周销',
                                              '品类',
                                              'Level',
                                              '状态变更日',
                                              '状态',
                                              '运营备注',
                                              '未售出周数',
                                              '注意事项']]

df_mrp = df_mrp[df_mrp['MRP SKU'] != ''].reset_index(drop=True)

df_mrp['自动周销'] = df_mrp['自动周销'].astype(float)
df_mrp['状态变更日'] = pd.to_datetime(df_mrp['状态变更日'])
df_mrp['未售出周数'] = df_mrp['未售出周数'].astype(int)

df_mrp['No.'] = [i+1 for i in range(len(df_mrp))]

df_mrp = df_mrp[['No.'] + df_mrp.columns.tolist()[:-1]]

df_mrp

,No.,MRP SKU,自动周销,品类,Level,状态变更日,状态,运营备注,未售出周数,注意事项
0,1,USOBDAD100-C,43.40,OBD,TOP20,2024-03-11,老品,,0,
1,2,USTCV17215HIE-2-C,40.50,TCV,TOP1,2021-07-19,老品,,0,
2,3,USCAK14152U-C,25.17,CA,TOP1,2023-01-02,老品,,0,
3,4,USOBDTK100,34.00,OBD,TOP10,2024-03-18,老品,,0,
4,5,USWS02632U-C,5.13,WS,TOP1,2023-04-10,老品,,0,
...,...,...,...,...,...,...,...,...,...,...
68657,68658,USWHN11008,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个
68658,68659,USWHN11288,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个
68659,68660,USWHN11236,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个
68660,68661,USWHS10547,0.00,WHS,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个


# 3. 在库在途计算

In [5]:
df_alternate = pd.read_excel(alternate,
                             header=0,
                             dtype=str).fillna('')

df_alternate['row'] = [i+1 for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USABS00021-C,USABS00021,,,,,,1
1,USABS00117B-C,USABS00117B,USABS00117,,,,,2
2,USABS00217B-C,USABS00217B,USABS00217,,,,,3
3,USABS00317B-C,USABS00317B,USABS00317,,,,,4
4,USABS00417-C,USABS00417,,,,,,5
...,...,...,...,...,...,...,...,...
48787,USWT80460,USET80460,,,,,,48788
48788,USWT90820,USET90820,,,,,,48789
48789,USWT91020,USET91020,,,,,,48790
48790,USWT91520,USET91520,,,,,,48791


In [6]:
list_df = []
for column in df_alternate.columns.tolist()[:-1]:
    df_temp =  df_alternate[['主料SKU', column, 'row']]
    df_temp.columns = ['主料', 'SKU', 'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_alternate = pd.concat(list_df, ignore_index=True)

df_alternate

,主料,SKU,row
0,USABS00021-C,USABS00021-C,1
1,USABS00117B-C,USABS00117B-C,2
2,USABS00217B-C,USABS00217B-C,3
3,USABS00317B-C,USABS00317B-C,4
4,USABS00417-C,USABS00417-C,5
...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299
106197,USIC02680L-6-C,USIC02680-6,31066
106198,USIC02680L-C,USIC02680,31067
106199,USIC2A366Z-C,USIC2A366,31174


In [7]:
df_price = pd.read_excel(price,
                         sheet_name=0,
                         header=0,
                         names=['SKU', '在途', '在库'],
                         usecols='A, E, F',
                         dtype=str).fillna({'SKU': '',
                                            '在途': 0,
                                            '在库': 0})

df_price = df_price[df_price['SKU'] != ''].reset_index(drop=True)

df_price['在库'] = df_price['在库'].astype(int)
df_price['在途'] = df_price['在途'].astype(int)

df_price['SKU'] = [sku.strip()[:-3] if sku.strip().endswith('-NC') else sku.strip()[:-2] if sku.strip().endswith('-N') else sku.strip()[:-2] if sku.strip().endswith('-D') else sku.strip() for sku in df_price['SKU'].tolist()]
df_price = df_price.groupby([df_price['SKU']], as_index=False).sum()

df_price = df_price[['SKU', '在库', '在途']]

df_price

,SKU,在库,在途
0,BCB96454-XH,0,0
1,BPD14040-XH,0,0
2,BPD8550086500+BRS31318,1,0
3,BRR34279-XH,0,0
4,CABM0021A,0,0
...,...,...,...
106030,YHTWS58202-Y-6,0,0
106031,YHTWS5W1ZBA-Y-10,0,0
106032,YHTWS5W1ZBA-Y-8,0,0
106033,YHTWS802AA-Y-6,0,0


In [8]:
print(df_alternate.columns)
print(df_price.columns)

df_temp_price_1 = pd.merge(left=df_alternate,
                           right=df_price,
                           how='left',
                           left_on='SKU',
                           right_on='SKU').fillna(0)


df_temp_price_1

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '在库', '在途'], dtype='object')


,主料,SKU,row,在库,在途
0,USABS00021-C,USABS00021-C,1,7.0,3.0
1,USABS00117B-C,USABS00117B-C,2,5.0,0.0
2,USABS00217B-C,USABS00217B-C,3,1.0,0.0
3,USABS00317B-C,USABS00317B-C,4,25.0,12.0
4,USABS00417-C,USABS00417-C,5,0.0,19.0
...,...,...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299,0.0,0.0
106197,USIC02680L-6-C,USIC02680-6,31066,0.0,0.0
106198,USIC02680L-C,USIC02680,31067,0.0,0.0
106199,USIC2A366Z-C,USIC2A366,31174,0.0,0.0


In [9]:
df_temp_price_1 = df_temp_price_1[['row', '在库', '在途']]
df_temp_price_1 = df_temp_price_1.groupby([df_temp_price_1['row']], as_index=False).sum()

df_temp_price_1

,row,在库,在途
0,1,8.0,3.0
1,2,5.0,0.0
2,3,9.0,0.0
3,4,36.0,12.0
4,5,5.0,19.0
...,...,...,...
48787,48788,6.0,2.0
48788,48789,26.0,6.0
48789,48790,9.0,2.0
48790,48791,3.0,0.0


In [10]:
print(df_alternate.columns)
print(df_temp_price_1.columns)

df_temp_price_1 = pd.merge(left=df_alternate,
                           right=df_temp_price_1,
                           how='left',
                           left_on='row',
                           right_on='row').fillna(0)

df_temp_price_1

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['row', '在库', '在途'], dtype='object')


,主料,SKU,row,在库,在途
0,USABS00021-C,USABS00021-C,1,8.0,3.0
1,USABS00117B-C,USABS00117B-C,2,5.0,0.0
2,USABS00217B-C,USABS00217B-C,3,9.0,0.0
3,USABS00317B-C,USABS00317B-C,4,36.0,12.0
4,USABS00417-C,USABS00417-C,5,5.0,19.0
...,...,...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299,22.0,25.0
106197,USIC02680L-6-C,USIC02680-6,31066,68.0,60.0
106198,USIC02680L-C,USIC02680,31067,2.0,10.0
106199,USIC2A366Z-C,USIC2A366,31174,13.0,0.0


In [11]:
print(df_price.columns)
print(df_alternate.columns)

df_temp_price_2 = pd.merge(left=df_price,
                           right=df_alternate,
                           how='left',
                           left_on='SKU',
                           right_on='SKU').fillna('')

df_temp_price_2 = df_temp_price_2[df_temp_price_2['row'] == '']
df_temp_price_2['主料'] = df_temp_price_2['SKU'].tolist()
df_temp_price_2 = df_temp_price_2[['主料', 'SKU', 'row', '在库', '在途']]

df_temp_price_2

Index(['SKU', '在库', '在途'], dtype='object')
Index(['主料', 'SKU', 'row'], dtype='object')


,主料,SKU,row,在库,在途
0,BCB96454-XH,BCB96454-XH,,0,0
1,BPD14040-XH,BPD14040-XH,,0,0
2,BPD8550086500+BRS31318,BPD8550086500+BRS31318,,1,0
3,BRR34279-XH,BRR34279-XH,,0,0
4,CABM0021A,CABM0021A,,0,0
...,...,...,...,...,...
106030,YHTWS58202-Y-6,YHTWS58202-Y-6,,0,0
106031,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,,0,0
106032,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,,0,0
106033,YHTWS802AA-Y-6,YHTWS802AA-Y-6,,0,0


In [12]:
df_price = pd.concat([df_temp_price_1, df_temp_price_2], ignore_index=True)
df_price = df_price[['SKU', '在库', '在途']]

df_price

,SKU,在库,在途
0,USABS00021-C,8.0,3.0
1,USABS00117B-C,5.0,0.0
2,USABS00217B-C,9.0,0.0
3,USABS00317B-C,36.0,12.0
4,USABS00417-C,5.0,19.0
...,...,...,...
133074,YHTWS58202-Y-6,0.0,0.0
133075,YHTWS5W1ZBA-Y-10,0.0,0.0
133076,YHTWS5W1ZBA-Y-8,0.0,0.0
133077,YHTWS802AA-Y-6,0.0,0.0


In [13]:
print(df_mrp.columns)
print(df_price.columns)

df_mrp = pd.merge(left=df_mrp,
                  right=df_price,
                  how='left',
                  left_on='MRP SKU',
                  right_on='SKU').fillna(0)

df_mrp['在库'] = df_mrp['在库'].astype(int)
df_mrp['在途'] = df_mrp['在途'].astype(int)

df_mrp = df_mrp[df_mrp.columns.tolist()[:-3] + ['在库', '在途']]

df_mrp

Index(['No.', 'MRP SKU', '自动周销', '品类', 'Level', '状态变更日', '状态', '运营备注', '未售出周数',
       '注意事项'],
      dtype='object')
Index(['SKU', '在库', '在途'], dtype='object')


,No.,MRP SKU,自动周销,品类,Level,状态变更日,状态,运营备注,未售出周数,注意事项,在库,在途
0,1,USOBDAD100-C,43.40,OBD,TOP20,2024-03-11,老品,,0,,173,229
1,2,USTCV17215HIE-2-C,40.50,TCV,TOP1,2021-07-19,老品,,0,,603,102
2,3,USCAK14152U-C,25.17,CA,TOP1,2023-01-02,老品,,0,,142,166
3,4,USOBDTK100,34.00,OBD,TOP10,2024-03-18,老品,,0,,261,0
4,5,USWS02632U-C,5.13,WS,TOP1,2023-04-10,老品,,0,,401,0
...,...,...,...,...,...,...,...,...,...,...,...,...
68657,68658,USWHN11008,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,0
68658,68659,USWHN11288,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,8
68659,68660,USWHN11236,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,5
68660,68661,USWHS10547,0.00,WHS,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,0


# 4. 海运计算

In [14]:
df_shipping = pd.read_excel(shipping,
                            sheet_name='US. 在途明细-总表',
                            header=0,
                            dtype=str).fillna('')[['SKU', '预报数量']]

df_shipping = df_shipping[df_shipping['SKU'] != ''].reset_index(drop=True)

df_shipping.columns = ['SKU', '近期海运上架数']
df_shipping['近期海运上架数'] = df_shipping['近期海运上架数'].astype(int)

df_shipping = df_shipping.groupby([df_shipping['SKU']], as_index=False).sum()

df_shipping

,SKU,近期海运上架数
0,USABS00021-C,3
1,USABS00317B-C,12
2,USABS00417-C,19
3,USABS00917-C,10
4,USABS01217-C,24
...,...,...
32199,USWSP87102B0-2-YC,6
32200,USWT03647,5
32201,USWT90020,6
32202,USWT90120,4


In [15]:
print(df_alternate.columns)
print(df_shipping.columns)

df_temp_shipping_1 = pd.merge(left=df_alternate,
                              right=df_shipping,
                              how='left',
                              left_on='SKU',
                              right_on='SKU').fillna(0)


df_temp_shipping_1

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '近期海运上架数'], dtype='object')


,主料,SKU,row,近期海运上架数
0,USABS00021-C,USABS00021-C,1,3.0
1,USABS00117B-C,USABS00117B-C,2,0.0
2,USABS00217B-C,USABS00217B-C,3,0.0
3,USABS00317B-C,USABS00317B-C,4,12.0
4,USABS00417-C,USABS00417-C,5,19.0
...,...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299,0.0
106197,USIC02680L-6-C,USIC02680-6,31066,0.0
106198,USIC02680L-C,USIC02680,31067,0.0
106199,USIC2A366Z-C,USIC2A366,31174,0.0


In [16]:
df_temp_shipping_1 = df_temp_shipping_1[['row', '近期海运上架数']]
df_temp_shipping_1 = df_temp_shipping_1.groupby([df_temp_shipping_1['row']], as_index=False).sum()

df_temp_shipping_1

,row,近期海运上架数
0,1,3.0
1,2,0.0
2,3,0.0
3,4,12.0
4,5,19.0
...,...,...
48787,48788,2.0
48788,48789,6.0
48789,48790,2.0
48790,48791,0.0


In [17]:
print(df_alternate.columns)
print(df_temp_shipping_1.columns)

df_temp_shipping_1 = pd.merge(left=df_alternate,
                              right=df_temp_shipping_1,
                              how='left',
                              left_on='row',
                              right_on='row').fillna(0)

df_temp_shipping_1

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['row', '近期海运上架数'], dtype='object')


,主料,SKU,row,近期海运上架数
0,USABS00021-C,USABS00021-C,1,3.0
1,USABS00117B-C,USABS00117B-C,2,0.0
2,USABS00217B-C,USABS00217B-C,3,0.0
3,USABS00317B-C,USABS00317B-C,4,12.0
4,USABS00417-C,USABS00417-C,5,19.0
...,...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299,25.0
106197,USIC02680L-6-C,USIC02680-6,31066,60.0
106198,USIC02680L-C,USIC02680,31067,10.0
106199,USIC2A366Z-C,USIC2A366,31174,0.0


In [18]:
print(df_shipping.columns)
print(df_alternate.columns)

df_temp_shipping_2 = pd.merge(left=df_shipping,
                              right=df_alternate,
                              how='left',
                              left_on='SKU',
                              right_on='SKU').fillna('')

df_temp_shipping_2 = df_temp_shipping_2[df_temp_shipping_2['row'] == '']
df_temp_shipping_2['主料'] = df_temp_shipping_2['SKU'].tolist()
df_temp_shipping_2 = df_temp_shipping_2[['主料', 'SKU', 'row', '近期海运上架数']]

df_temp_shipping_2

Index(['SKU', '近期海运上架数'], dtype='object')
Index(['主料', 'SKU', 'row'], dtype='object')


,主料,SKU,row,近期海运上架数
13,USABS05515,USABS05515,,4
26,USABS10040,USABS10040,,6
55,USABS11830,USABS11830,,10
57,USABS12120,USABS12120,,1
85,USABS14304,USABS14304,,2
...,...,...,...,...
32199,USWSP87102B0-2-YC,USWSP87102B0-2-YC,,6
32200,USWT03647,USWT03647,,5
32201,USWT90020,USWT90020,,6
32202,USWT90120,USWT90120,,4


In [19]:
df_shipping = pd.concat([df_temp_shipping_1, df_temp_shipping_2], ignore_index=True)
df_shipping = df_shipping[['SKU', '近期海运上架数']]

df_shipping

,SKU,近期海运上架数
0,USABS00021-C,3.0
1,USABS00117B-C,0.0
2,USABS00217B-C,0.0
3,USABS00317B-C,12.0
4,USABS00417-C,19.0
...,...,...
117165,USWSP87102B0-2-YC,6.0
117166,USWT03647,5.0
117167,USWT90020,6.0
117168,USWT90120,4.0


In [20]:
print(df_mrp.columns)
print(df_shipping.columns)

df_mrp = pd.merge(left=df_mrp,
                  right=df_shipping,
                  how='left',
                  left_on='MRP SKU',
                  right_on='SKU').fillna(0)

df_mrp['近期海运上架数'] = df_mrp['近期海运上架数'].astype(int)

df_mrp = df_mrp[df_mrp.columns.tolist()[:-2] + ['近期海运上架数']]

df_mrp

Index(['No.', 'MRP SKU', '自动周销', '品类', 'Level', '状态变更日', '状态', '运营备注', '未售出周数',
       '注意事项', '在库', '在途'],
      dtype='object')
Index(['SKU', '近期海运上架数'], dtype='object')


,No.,MRP SKU,自动周销,品类,Level,状态变更日,状态,运营备注,未售出周数,注意事项,在库,在途,近期海运上架数
0,1,USOBDAD100-C,43.40,OBD,TOP20,2024-03-11,老品,,0,,173,229,119
1,2,USTCV17215HIE-2-C,40.50,TCV,TOP1,2021-07-19,老品,,0,,603,102,102
2,3,USCAK14152U-C,25.17,CA,TOP1,2023-01-02,老品,,0,,142,166,144
3,4,USOBDTK100,34.00,OBD,TOP10,2024-03-18,老品,,0,,261,0,0
4,5,USWS02632U-C,5.13,WS,TOP1,2023-04-10,老品,,0,,401,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68657,68658,USWHN11008,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,0,0
68658,68659,USWHN11288,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,8,8
68659,68660,USWHN11236,0.00,WHN,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,5,5
68660,68661,USWHS10547,0.00,WHS,普通,2024-02-28,清仓,,0,WHS/WHN 不发单个,0,0,0


In [21]:
from datetime import datetime

In [22]:
df_mrp['在库+海运上架数'] = [df_mrp.loc[i, '在库'] + df_mrp.loc[i, '近期海运上架数'] for i in range(len(df_mrp))]

df_mrp = df_mrp[df_mrp['状态'] != '清仓'].reset_index(drop=True)
df_mrp = df_mrp[~((df_mrp['状态'] == '新品') & (df_mrp['在库'] == 0))].reset_index(drop=True)
df_mrp['No.'] = [i+1 for i in range(len(df_mrp))]

df_mrp.to_excel('./统计源_' + datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_mrp

,No.,MRP SKU,自动周销,品类,Level,状态变更日,状态,运营备注,未售出周数,注意事项,在库,在途,近期海运上架数,在库+海运上架数
0,1,USOBDAD100-C,43.40,OBD,TOP20,2024-03-11,老品,,0,,173,229,119,292
1,2,USTCV17215HIE-2-C,40.50,TCV,TOP1,2021-07-19,老品,,0,,603,102,102,705
2,3,USCAK14152U-C,25.17,CA,TOP1,2023-01-02,老品,,0,,142,166,144,286
3,4,USOBDTK100,34.00,OBD,TOP10,2024-03-18,老品,,0,,261,0,0,261
4,5,USWS02632U-C,5.13,WS,TOP1,2023-04-10,老品,,0,,401,0,0,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66131,66132,USHCR00013-C,0.00,HCR,普通,2022-10-24,老品,,12,,5,0,0,5
66132,66133,USHCR94762-C,0.00,HCR,普通,2024-02-19,老品,,12,,6,3,3,9
66133,66134,USHCR99202-C,0.12,HCR,普通,2024-02-05,老品,,7,,3,6,6,9
66134,66135,USHCR98032-C,0.00,HCR,普通,2022-10-03,老品,,12,,15,0,0,15
